In [ ]:
import os
import sys

sys.path.append('../source')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np

from dftd3.interface import Structure, RationalDampingParam, DispersionModel
from Utilities import H_TO_KJ, BOHR_TO_ANGSTROM, to_Z

In [ ]:
def calculate_d3_dimer(coords_1, coords_2, elements_1, elements_2, functional='pbe0', damping_params=None):
    elements_1, elements_2 = to_Z(elements_1), to_Z(elements_2)
    energy_monomer_1, energy_monomer_2, energy_dimer = [], [], []
    if damping_params is None:
        damping_params = RationalDampingParam(method=functional)
    for monomer_1 in coords_1:
        model = DispersionModel(elements_1, monomer_1 / BOHR_TO_ANGSTROM)
        e_1 = model.get_dispersion(damping_params, grad=False)['energy']
        energy_monomer_1.append(e_1)

    for monomer_2 in coords_2:
        model = DispersionModel(elements_2, monomer_2 / BOHR_TO_ANGSTROM)
        e_2 = model.get_dispersion(damping_params, grad=False)['energy']
        energy_monomer_2.append(e_2)

    dimer_coords = np.concatenate((coords_1, coords_2), axis=1)
    dimer_elements = np.concatenate((elements_1, elements_2))
    for dimer in dimer_coords:
        model = DispersionModel(dimer_elements, dimer / BOHR_TO_ANGSTROM)
        e_dimer = model.get_dispersion(damping_params, grad=False)['energy']
        energy_dimer.append(e_dimer)

    energy_monomer_1, energy_monomer_2, energy_dimer = np.hstack(energy_monomer_1), np.hstack(energy_monomer_2), np.hstack(energy_dimer)
    return np.float32((energy_dimer - energy_monomer_1 - energy_monomer_2) * H_TO_KJ)

In [ ]:
DATA = np.load('DES5M.npy', allow_pickle=True).item()

In [ ]:
for functional in ['PBE0', 'TPSS', 'BP86', 'TPSSh']:
    D3 = {}
    for key in DATA:
        D3[key] = calculate_d3_dimer(*DATA[key]['coordinates'], *DATA[key]['elements'], functional=functional)
    np.save(f'DES5M_D3_{functional}', D3)

In [ ]:
for custom_params in [(0.45, 1.8, 4.5)]:
    damping_params = RationalDampingParam(a1=custom_params[0], s8=custom_params[1], a2=custom_params[2])
    D3 = {}
    for key in DATA:
        D3[key] = calculate_d3_dimer(*DATA[key]['coordinates'], *DATA[key]['elements'], damping_params=damping_params)
    np.save(f'DES5M_D3_{int(100 * custom_params[0])}_{int(100 * custom_params[1])}_{int (100 * custom_params[2])}', D3)

In [ ]:
len(D3), len(DATA)